In [10]:
!pip install vosk

Defaulting to user installation because normal site-packages is not writeable


In [12]:
!pip install gtts pygame

Defaulting to user installation because normal site-packages is not writeable


<h1>TRANSCRIPCIÓN DE VOZ A TEXTO / PARA LLENAR UNA PLANTILLA EN PDF.</h1>

**<h2>Universidad Nacional de Loja</h2>**

----

 **<h3>Materia:** Human Perception in Computer Vision</h3>
  **<h3>Integrantes:**</h3>
+ Luis Bermeo
+ Anthony Luzuriaga
+ Gerar Quizhpe
+ Alex Viñamagua

In [36]:
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'FICHA DE DESCRIPCIÓN DE AFLORAMIENTOS', 0, 1, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')

    def chapter_body(self, body):
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, body)
        self.ln()

    def table(self, data):
        self.set_font('Arial', 'B', 12)
        # Header, el total es de 190 el largo del documento. osea 19cm.
        self.cell(40, 10, 'Responsable:', 1)
        self.set_font('Arial', '', 12)
        self.cell(150, 10, data['responsable'], 1)
        self.ln()

        self.set_font('Arial', 'B', 12)
        self.cell(40, 10, 'Afloramiento N°:', 1)
        self.set_font('Arial', '', 12)
        self.cell(150, 10, data['afloramiento'], 1)
        self.ln()

        self.set_font('Arial', 'B', 12)
        self.cell(40, 10, 'Carta Topográfica:', 1)
        self.set_font('Arial', '', 12)
        self.cell(150, 10, data['carta_topografica'], 1)
        self.ln()
        
        ######################################################################
        self.set_font('Arial', 'B', 12)
        self.cell(190, 10, 'UBICACIÓN', 1, 1, 'C')

        self.cell(30, 10, 'Provincia:', 1)
        self.set_font('Arial', '', 12)
        self.cell(160, 10, data['provincia'], 1)
        self.ln()

        self.set_font('Arial', 'B', 12)
        self.cell(30, 10, 'Cantón:', 1)
        self.set_font('Arial', '', 12)
        self.cell(160, 10, data['canton'], 1)
        self.ln()

        self.set_font('Arial', 'B', 12)
        self.cell(30, 10, 'Parroquia:', 1)
        self.set_font('Arial', '', 12)
        self.cell(160, 10, data['parroquia'], 1)
        self.ln()

        self.set_font('Arial', 'B', 12)
        self.cell(30, 10, 'Sector:', 1)
        self.set_font('Arial', '', 12)
        self.cell(160, 10, data['sector'], 1)
        self.ln()

        self.set_font('Arial', 'B', 12)
        self.cell(30, 10, 'DATUM:', 1)
        self.set_font('Arial', '', 12)
        self.cell(160, 10, "WGS 1984", 1)
        self.ln()
        #######################################################################
        self.set_font('Arial', 'B', 12)
        self.cell(190, 10, 'Localización', 1, 1, 'C')

        self.cell(30, 10, 'X:', 1)
        self.set_font('Arial', '', 12)
        self.cell(160, 10, data['localizacion_x'], 1)
        self.ln()

        self.set_font('Arial', 'B', 12)
        self.cell(30, 10, 'Y:', 1)
        self.set_font('Arial', '', 12)
        self.cell(160, 10, data['localizacion_y'], 1)
        self.ln()

        self.set_font('Arial', 'B', 12)
        self.cell(30, 10, 'Z:', 1)
        self.set_font('Arial', '', 12)
        self.cell(160, 10, str(data['localizacion_z'])+" m", 1)
        self.ln()
        ########################################################################
        self.set_font('Arial', 'B', 12)
        self.cell(190, 10, 'Tipo de afloramiento', 1, 1, 'C')

        self.cell(45, 10, 'Alforamiento Natural:', 1)
        self.set_font('Arial', '', 12)
        self.cell(45, 10, data['tipo_afloramiento_natural'], 1)

        self.set_font('Arial', 'B', 12)
        self.cell(60, 10, 'Alforamiento Antropogenico:', 1)
        self.set_font('Arial', '', 12)
        self.cell(40, 10, data['tipo_afloramiento_antropogenico'], 1)
        self.ln()
        ########################################################################
        self.set_font('Arial', 'B', 12)
        self.cell(190, 10, 'Dimensiones', 1, 1, 'C')

        self.cell(20, 10, 'Largo:', 1)
        self.set_font('Arial', '', 12)
        self.cell(75, 10, str(data['largo'])+" m",1)

        self.set_font('Arial', 'B', 12)
        self.cell(20, 10, 'Ancho:', 1)
        self.set_font('Arial', '', 12)
        self.cell(75, 10, str(data['ancho'])+" m", 1)
        self.ln()

In [41]:
#Generar Word

from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

def create_word(data):
    doc = Document()

    # Header
    header = doc.add_heading(level=1)
    header_run = header.add_run('FICHA DE DESCRIPCIÓN DE AFLORAMIENTOS')
    header_run.font.bold = True
    header_run.font.size = Pt(12)
    header.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    # Function to add table rows
    def add_table_row(table, cells):
        row_cells = table.add_row().cells
        for idx, cell_text in enumerate(cells):
            row_cells[idx].text = cell_text
    
    # Function para cambiar el color del texto 
    def add_bold_heading(doc, text):
        paragraph = doc.add_heading(level=2)
        run = paragraph.add_run(text)
        run.font.bold = True
        run.font.color.rgb = RGBColor(0, 0, 0)  # Negro

    # Table 1: General Information
    table = doc.add_table(rows=0, cols=2)
    table.style = 'Table Grid'
    add_table_row(table, ['Responsable:', data['responsable']])
    add_table_row(table, ['Afloramiento N°:', data['afloramiento']])
    add_table_row(table, ['Carta Topográfica:', data['carta_topografica']])

    # Table 2: Ubicación
    doc.add_heading('UBICACIÓN', level=2)
    table = doc.add_table(rows=0, cols=2)
    table.style = 'Table Grid'
    add_table_row(table, ['Provincia:', data['provincia']])
    add_table_row(table, ['Cantón:', data['canton']])
    add_table_row(table, ['Parroquia:', data['parroquia']])
    add_table_row(table, ['Sector:', data['sector']])
    add_table_row(table, ['DATUM:', data['datum']])

    # Table 3: Localización
    doc.add_heading('Localización', level=2)
    table = doc.add_table(rows=0, cols=2)
    table.style = 'Table Grid'
    add_table_row(table, ['X:', data['localizacion_x']])
    add_table_row(table, ['Y:', data['localizacion_y']])
    add_table_row(table, ['Z:', data['localizacion_z']])

    # Table 4: Tipo de afloramiento
    doc.add_heading('Tipo de afloramiento', level=2)
    table = doc.add_table(rows=0, cols=4)
    table.style = 'Table Grid'
    add_table_row(table, ['Alforamiento Natural:', data['tipo_afloramiento_natural'], 'Alforamiento Antropogenico:', data['tipo_afloramiento_antropogenico']])

    # Table 5: Dimensiones
    doc.add_heading('Dimensiones', level=2)
    table = doc.add_table(rows=0, cols=4)
    table.style = 'Table Grid'
    add_table_row(table, ['Largo:', data['largo'], 'Ancho:', data['ancho']])

    # Save the document
    output_path = 'plantilla_ejemplo_datos.docx'
    doc.save(output_path)
    return output_path

In [37]:
# Función para crear y guardar el PDF
def create_pdf(data):
    pdf = PDF()
    pdf.add_page()
    pdf.table(data)
    output_path = "plantilla_ejemplo_datos.pdf"
    pdf.output(output_path)
    return output_path

In [6]:
import os
import wave
import json
import vosk
import pyaudio
import time
import pygame
from gtts import gTTS

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
# Inicializar Pygame
pygame.mixer.init()

is_recording = False

In [8]:
def play_sound(file):
    pygame.mixer.music.load(file)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)

In [9]:
def text_to_audio(text, filename):
    tts = gTTS(text=text, lang='es')
    tts.save(filename)

In [10]:
import tempfile

def play_text(text):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio:
        temp_filename = temp_audio.name
        text_to_audio(text, temp_filename)
        #tts = gTTS(text=text, lang='es')
        #tts.save(temp_filename)
    
    # Asegurarse de que el archivo se ha guardado antes de cargarlo
    if os.path.exists(temp_filename):
        pygame.mixer.music.load(temp_filename)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)
        time.sleep(0.5)  # Pequeño retraso para asegurarse de que Pygame ha terminado de usar el archivo
        #try:
        #    os.remove(temp_filename)
        #except PermissionError:
        #    print(f"Error: No se pudo eliminar el archivo {temp_filename}")
    else:
        print(f"Error: No se pudo encontrar el archivo {temp_filename}")

In [11]:
def convertir_a_numero(texto):
    # Diccionarios para mapeo de palabras a números
    unidades = {
        'uno': 1, 'un': 1, 'dos': 2, 'tres': 3, 'cuatro': 4, 'cinco': 5,
        'seis': 6, 'siete': 7, 'ocho': 8, 'nueve': 9, 'diez': 10,
        'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quince': 15,
        'dieciséis': 16, 'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19, 'veinte': 20,
        'veintiuno': 21, 'veintidós': 22, 'veintitrés': 23, 'veinticuatro': 24, 'veinticinco': 25,
        'veintiséis': 26, 'veintisiete': 27, 'veintiocho': 28, 'veintinueve': 29
    }
    decenas = {
        'treinta': 30, 'cuarenta': 40, 'cincuenta': 50, 'sesenta': 60,
        'setenta': 70, 'ochenta': 80, 'noventa': 90
    }
    centenas = {
        'ciento': 100, 'doscientos': 200, 'trescientos': 300, 'cuatrocientos': 400, 'quinientos': 500,
        'seiscientos': 600, 'setecientos': 700, 'ochocientos': 800, 'novecientos': 900
    }
    especiales = {
        'cien': 100, 'mil': 1000, 'millón': 1000000, 'millones': 1000000, 'cero': 0
    }

    palabras = texto.split()
    total = 0
    temporal = 0
    resultado = ""
    decimal_part = False

    for palabra in palabras:
        if palabra in unidades:
            if decimal_part:
                resultado += str(unidades[palabra])
            else:
                temporal += unidades[palabra]
        elif palabra in decenas:
            temporal += decenas[palabra]
        elif palabra in centenas:
            temporal += centenas[palabra]
        elif palabra in especiales:
            if palabra == 'mil':
                if temporal == 0:
                    temporal = 1
                total += temporal * 1000
                temporal = 0
            elif palabra == 'millón' or palabra == 'millones':
                if temporal == 0:
                    temporal = 1
                total += temporal * 1000000
                temporal = 0
            else:
                temporal += especiales[palabra]
        elif palabra == 'punto' or palabra == 'coma':  # Manejar separadores decimales
            if not decimal_part:
                resultado += str(total + temporal) + "."
                decimal_part = True
                temporal = 0
        elif palabra == 'y':
            continue
        else:
            return "0"

    if not decimal_part:
        resultado += str(total + temporal)
    return resultado

In [12]:
# Función para capturar voz, modelos en : https://alphacephei.com/vosk/models
def capture_audio():
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=8192)
    stream.start_stream()
    
    model = vosk.Model("model/vosk-model-small-es-0.42")
    rec = vosk.KaldiRecognizer(model, 16000)
    
    print("Di tu comando:")
    #play_text("Di tu comando")
    
    result_text = ""
    while True:
        # Lee datos del flujo de audio
        data = stream.read(4096)
        # Procesa los datos de audio y verifica si se ha reconocido una forma de onda completa
        if rec.AcceptWaveform(data):
            # Si se reconoce una forma de onda completa, obtiene el resultado en formato JSON
            result = rec.Result()
            text = json.loads(result)["text"]
            result_text += text + " "
            break

    stream.stop_stream()
    stream.close()
    p.terminate()
 
    return result_text.strip()

In [38]:
# Función para asignar valores a variables
def assign_variable(variable_name):
    
    prompt = f"Por favor, di el valor para {variable_name}:"
    #play_sound(f"audio/{variable_name}.mp3")
    play_text(prompt)
    
    value = capture_audio()
    
    captured_message = f"Capturado: {value}"
    #play_sound("audio/capturado.mp3")
    play_text(captured_message)
    
    if variable_name in ['afloramiento', 'localizacion_x', 'localizacion_y', 'localizacion_z', 'largo', 'ancho']:
        value = convertir_a_numero(value)
    
        # Manejar el caso especial para el tipo de afloramiento
    if variable_name in ['tipo_afloramiento_natural', 'tipo_afloramiento_antropogenico']:
        if value.lower() == 'si':
            value = 'X'
        else:
            value = ''
            
    data[variable_name] = value

In [14]:
def update_data(field, value):
    data[field] = value

In [30]:
# Diccionario para guardar variables
data = {
    'responsable': '',
    'afloramiento': '',
    'carta_topografica': '',
    'provincia': '',
    'canton': '',
    'parroquia': '',
    'sector': '',
    'localizacion_x': '',
    'localizacion_y': '',
    'localizacion_z': '',
    'tipo_afloramiento_natural': '',
    'tipo_afloramiento_antropogenico': '',
    'largo': '',
    'ancho': '',
}

In [49]:
def main():
    for key in data.keys():
        assign_variable(key)

In [20]:
# Generar Archivos
def generar_archivos():
    output_path_pdf = create_pdf(data)
    output_path_word = create_word(data)
    return output_path_pdf, output_path_word


In [32]:
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            start_btns = [gr.Button(f"Iniciar grabación {key}") for key in data.keys()]
            pause_btn = gr.Button("Pausar")
            stop_btn = gr.Button("Detener")
        with gr.Column():
            gr.Markdown("Formulario de la data")
            responsable = gr.Textbox(label="Responsable")
            afloramiento = gr.Textbox(label="Afloramiento N°")
            carta_topografica = gr.Textbox(label="Carta Topográfica")
            provincia = gr.Textbox(label="Provincia")
            canton = gr.Textbox(label="Cantón")
            parroquia = gr.Textbox(label="Parroquia")
            sector = gr.Textbox(label="Sector")
            localizacion_x = gr.Textbox(label="Localización X")
            localizacion_y = gr.Textbox(label="Localización Y")
            localizacion_z = gr.Textbox(label="Localización Z")
            tipo_afloramiento_natural = gr.Textbox(label="Tipo de afloramiento natural")
            tipo_afloramiento_antropogenico = gr.Textbox(label="Tipo de afloramiento antropogénico")
            largo = gr.Textbox(label="Largo")
            ancho = gr.Textbox(label="Ancho")
            generate_pdf_btn = gr.Button("Generar PDF y Word")

            textboxes = {
                'responsable': responsable,
                'afloramiento': afloramiento,
                'carta_topografica': carta_topografica,
                'provincia': provincia,
                'canton': canton,
                'parroquia': parroquia,
                'sector': sector,
                'localizacion_x': localizacion_x,
                'localizacion_y': localizacion_y,
                'localizacion_z': localizacion_z,
                'tipo_afloramiento_natural': tipo_afloramiento_natural,
                'tipo_afloramiento_antropogenico': tipo_afloramiento_antropogenico,
                'largo': largo,
                'ancho': ancho,
            }

    for key, btn in zip(data.keys(), start_btns):
        btn.click(lambda k=key: assign_variable(k), inputs=None, outputs=textboxes[key])

    generate_pdf_btn.click(generar_archivos)

In [40]:
if __name__ == "__main__":
    #interfaz_ui.launch(server_port=7869)
    demo.launch(server_port=7869)
    #demo.close()
    #main()

Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


Di tu comando:
Di tu comando:
Di tu comando:
Di tu comando:


Traceback (most recent call last):
  File "C:\Users\Acer\AppData\Roaming\Python\Python311\site-packages\gradio\queueing.py", line 532, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Acer\AppData\Roaming\Python\Python311\site-packages\gradio\route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Acer\AppData\Roaming\Python\Python311\site-packages\gradio\blocks.py", line 1928, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Acer\AppData\Roaming\Python\Python311\site-packages\gradio\blocks.py", line 1514, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\anyio\to_thread.py", line 28, in run_sync
    return await get_asyn